# PCA-based Heart Model to Image Registration Experiment

This notebook demonstrates using the `RegisterModelToImagePCA` class to register
a statistical shape model to patient CT images using PCA-based shape variation.

## Overview
- Uses the KCL Heart Model PCA statistical shape model
- Registers to the same Duke Heart CT data as the original notebook
- Two-stage optimization: rigid alignment + PCA shape fitting
- Converts segmentation mask to intensity image for registration

## Setup and Imports

In [1]:
# PCA-based Heart Model to Image Registration Experiment

import os
from pathlib import Path

import itk
import numpy as np
import pyvista as pv
from itk import TubeTK as ttk

# Import from PhysioMotion4D package
from physiomotion4d import (
    ContourTools,
    RegisterModelsICP,
    RegisterModelsPCA,
    TransformTools,
)

## Define File Paths

In [2]:
# Patient CT image (defines coordinate frame)
patient_data_dir = Path.cwd().parent.parent / "data" / "Slicer-Heart-CT"
patient_ct_path = patient_data_dir / "patient_img.mha"
patient_ct_heart_mask_path = patient_data_dir / "patient_heart_wall_mask.nii.gz"

# PCA heart model data
heart_model_data_dir = Path.cwd().parent.parent / "data" / "KCL-Heart-Model"
heart_model_path = heart_model_data_dir / "average_mesh.vtk"

# PCA heart model data
template_model_data_dir = Path.cwd().parent.parent / "data" / "KCL-Heart-Model" / "pca"
template_model_surface_path = template_model_data_dir / "pca_All_mean.vtk"
pca_json_path = template_model_data_dir / "pca.json"
pca_group_key = "All"

# Output directory
output_dir = Path.cwd() / "results_pca"
os.makedirs(output_dir, exist_ok=True)

print(f"Patient data: {patient_data_dir}")
print(f"PCA Model data: {template_model_data_dir}")
print(f"Output directory: {output_dir}")

Patient data: C:\src\Projects\PhysioMotion\physiomotion4d\data\Slicer-Heart-CT
PCA Model data: C:\src\Projects\PhysioMotion\physiomotion4d\data\KCL-Heart-Model\pca
Output directory: C:\src\Projects\PhysioMotion\physiomotion4d\experiments\Heart-Model_To_Patient\results_pca


## Load and Preprocess Patient Image

In [3]:
# Load patient CT image
print("Loading patient CT image...")
patient_image = itk.imread(str(patient_ct_path))
print(f"  Original size: {itk.size(patient_image)}")
print(f"  Original spacing: {itk.spacing(patient_image)}")

# Resample to 1mm isotropic spacing
print("Resampling to sotropic...")
resampler = ttk.ResampleImage.New(Input=patient_image)
resampler.SetMakeHighResIso(True)
resampler.Update()
patient_image = resampler.GetOutput()

print(f"  Resampled size: {itk.size(patient_image)}")
print(f"  Resampled spacing: {itk.spacing(patient_image)}")

# Save preprocessed image
itk.imwrite(patient_image, str(output_dir / "patient_image.mha"), compression=True)
print("✓ Saved preprocessed image")

Loading patient CT image...
  Original size: itkSize3 ([307, 234, 152])
  Original spacing: itkVectorD3 ([1, 1, 1])
Resampling to sotropic...


  Resampled size: itkSize3 ([307, 234, 152])
  Resampled spacing: itkVectorD3 ([1, 1, 1])
✓ Saved preprocessed image


## Load and Process Heart Segmentation Mask

In [4]:
# Load heart segmentation mask
print("Loading heart segmentation mask...")
patient_heart_mask_image = itk.imread(str(patient_ct_heart_mask_path))

print(f"  Mask size: {itk.size(patient_heart_mask_image)}")
print(f"  Mask spacing: {itk.spacing(patient_heart_mask_image)}")

Loading heart segmentation mask...
  Mask size: itkSize3 ([307, 234, 152])
  Mask spacing: itkVectorD3 ([1, 1, 1])


In [5]:
# Handle image orientation (flip if needed)
flip0 = np.array(patient_heart_mask_image.GetDirection())[0, 0] < 0
flip1 = np.array(patient_heart_mask_image.GetDirection())[1, 1] < 0
flip2 = np.array(patient_heart_mask_image.GetDirection())[2, 2] < 0

if flip0 or flip1 or flip2:
    print(f"Flipping image axes: {flip0}, {flip1}, {flip2}")

    # Flip CT image
    flip_filter = itk.FlipImageFilter.New(Input=patient_image)
    flip_filter.SetFlipAxes([int(flip0), int(flip1), int(flip2)])
    flip_filter.SetFlipAboutOrigin(True)
    flip_filter.Update()
    patient_image = flip_filter.GetOutput()
    id_mat = itk.Matrix[itk.D, 3, 3]()
    id_mat.SetIdentity()
    patient_image.SetDirection(id_mat)

    # Flip mask image
    flip_filter = itk.FlipImageFilter.New(Input=patient_heart_mask_image)
    flip_filter.SetFlipAxes([int(flip0), int(flip1), int(flip2)])
    flip_filter.SetFlipAboutOrigin(True)
    flip_filter.Update()
    patient_heart_mask_image = flip_filter.GetOutput()
    patient_heart_mask_image.SetDirection(id_mat)

    print("✓ Images flipped to standard orientation")

# Save oriented images
itk.imwrite(
    patient_image, str(output_dir / "patient_image_oriented.mha"), compression=True
)
itk.imwrite(
    patient_heart_mask_image,
    str(output_dir / "patient_heart_mask_oriented.mha"),
    compression=True,
)

Flipping image axes: True, True, False
✓ Images flipped to standard orientation


__array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
__array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
__array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword


## Convert Segmentation Mask to a Surface

In [6]:
contour_tools = ContourTools()
patient_surface = contour_tools.extract_contours(patient_heart_mask_image)

## Perform Initial ICP Rigid Registration

Use ICP (Iterative Closest Point) with affine mode to align the model surface to the patient surface extracted from the segmentation mask. This provides a good initial alignment for the PCA-based registration.

The ICP registration pipeline:
1. Centroid alignment (automatic)
2. Rigid ICP alignment

The PCA registration will then refine this initial alignment with shape model constraints.

In [7]:
# Load the pca model
print("Loading PCA heart model...")
template_model = pv.read(str(heart_model_path))

template_model_surface = pv.read(template_model_surface_path)
print(f"  Template surface: {template_model_surface.n_points} points")

icp_registrar = RegisterModelsICP(fixed_model=patient_surface)

icp_result = icp_registrar.register(
    transform_type="Affine", moving_model=template_model_surface, max_iterations=2000
)

# Get the aligned mesh and transform
icp_registered_model_surface = icp_result["registered_model"]
icp_forward_point_transform = icp_result["forward_point_transform"]

print("\n✓ ICP affine registration complete")
print("   Transform =", icp_result["forward_point_transform"])

# Save aligned model
icp_registered_model_surface.save(str(output_dir / "icp_registered_model_surface.vtp"))
print("  Saved ICP-aligned model surface")

itk.transformwrite(
    [icp_result["forward_point_transform"]],
    str(output_dir / "icp_transform.hdf"),
    compression=True,
)
print("  Saved ICP transform")

Loading PCA heart model...


2026-01-25 18:14:20 INFO RegisterModelsICP ======================================================================


2026-01-25 18:14:20 INFO RegisterModelsICP AFFINE ICP Alignment


2026-01-25 18:14:20 INFO RegisterModelsICP ======================================================================


2026-01-25 18:14:20 INFO RegisterModelsICP Step 1: Translating by [648.62508965 318.68302345 962.48420525] to align centroids...


...

2026-01-25 18:14:22 INFO RegisterModelsICP Step 2: Performing rigid ICP (max iterations: 2000)...


2026-01-25 18:14:31 INFO RegisterModelsICP Step 3: Performing affine ICP (max iterations: 2000)...


2026-01-25 18:14:38 INFO RegisterModelsICP AFFINE ICP registration complete!



✓ ICP affine registration complete
   Transform = AffineTransform (000001751E2376E0)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 1
  Modified Time: 1622
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.987866 -0.0938189 -0.132732 
    0.0351459 0.871548 0.0948309 
    0.0471311 -0.0327586 0.795693 
  Offset: [648.613, 318.376, 960.745]
  Center: [0, 0, 0]
  Translation: [648.613, 318.376, 960.745]
  Inverse: 
    1.00092 0.113513 0.153439 
    -0.0337609 1.13844 -0.141311 
    -0.0606774 0.0401457 1.24186 
  Singular: 0



  Saved ICP-aligned model surface
  Saved ICP transform


In [8]:
# Apply ICP transform to the full average mesh (not just surface)
# This gets the volumetric mesh into patient space for PCA registration
transform_tools = TransformTools()
icp_registered_model = transform_tools.transform_pvcontour(
    template_model, icp_forward_point_transform
)
icp_registered_model.save(str(output_dir / "icp_registered_model.vtk"))
print("\n✓ Applied ICP transform to full model mesh")


✓ Applied ICP transform to full model mesh


## Initialize PCA Registration

In [9]:
## Initialize PCA Registration
print("=" * 70)

# Use the ICP-aligned mesh as the starting point for PCA registration
pca_registrar = RegisterModelsPCA.from_slicersalt(
    pca_template_model=template_model_surface,
    pca_json_filename=pca_json_path,
    pca_group_key=pca_group_key,
    pca_number_of_modes=10,
    post_pca_transform=icp_forward_point_transform,
    fixed_model=patient_surface,
    reference_image=patient_image,
)

itk.imwrite(pca_registrar.fixed_distance_map, str(output_dir / "distance_map.mha"))

print("✓ PCA registrar initialized")
print("  Using ICP-aligned mesh as starting point")
print(f"  Number of points: {len(pca_registrar.pca_template_model.points)}")
print(f"  Number of PCA modes: {pca_registrar.pca_number_of_modes}")

2026-01-25 18:14:43 INFO Loading PCA data from SlicerSALT format...


2026-01-25 18:14:43 INFO   JSON file: C:\src\Projects\PhysioMotion\physiomotion4d\data\KCL-Heart-Model\pca\pca.json


2026-01-25 18:14:43 INFO   Group key: All


2026-01-25 18:14:43 INFO Reading JSON file...


2026-01-25 18:14:49 INFO   Loaded 20 standard deviations


...

2026-01-25 18:14:49 INFO SlicerSALT PCA data loaded successfully!


2026-01-25 18:14:49 INFO ContourTools Computing signed distance map...


2026-01-25 18:14:52 INFO RegisterModelsPCA Converting mean shape points to ITK format...


2026-01-25 18:14:53 INFO RegisterModelsPCA   Converted 167240 points to ITK format


✓ PCA registrar initialized
  Using ICP-aligned mesh as starting point
  Number of points: 167240
  Number of PCA modes: 10


## Run PCA-Based Shape Optimization

Now that we have a good initial alignment from ICP affine registration, we run the PCA-based registration to optimize the shape parameters.

In [10]:
print("\n" + "=" * 70)
print("PCA-BASED SHAPE OPTIMIZATION")
print("=" * 70)
print("\nRunning complete PCA registration pipeline...")
print("  (Starting from ICP-aligned mesh)")

result = pca_registrar.register(
    pca_number_of_modes=10,  # Use first 10 PCA modes
)

pca_registered_model_surface = result["registered_model"]

print("\n✓ PCA registration complete")

2026-01-25 18:14:53 INFO RegisterModelsPCA ======================================================================


2026-01-25 18:14:53 INFO RegisterModelsPCA PCA-BASED MODEL-TO-IMAGE REGISTRATION


2026-01-25 18:14:53 INFO RegisterModelsPCA ======================================================================


2026-01-25 18:14:53 INFO RegisterModelsPCA Number of points: 167240


2026-01-25 18:14:53 INFO RegisterModelsPCA Modes to use: 10


...

2026-01-25 18:15:59 INFO RegisterModelsPCA Transforming points: 133793/167240 (80.0%)


2026-01-25 18:16:00 INFO RegisterModelsPCA Transforming points: 150517/167240 (90.0%)


2026-01-25 18:16:01 INFO RegisterModelsPCA Transforming points: 167240/167240 (100.0%)


2026-01-25 18:16:01 INFO RegisterModelsPCA Registered model created with 167240 points



✓ PCA registration complete


### Display Registration Results

Review the optimization results from the PCA registration pipeline.


In [11]:
print("\n" + "=" * 70)
print("REGISTRATION RESULTS")
print("=" * 70)

# Display results
print("\nFinal Registration Metrics:")
print(f"  Final mean intensity: {result['mean_distance']:.4f}")

print("\nOptimized PCA Coefficients (in units of std deviations):")
for i, coef in enumerate(result["pca_coefficients"]):
    print(f"  Mode {i + 1:2d}: {coef:7.4f}")

print("\n✓ Registration pipeline complete!")


REGISTRATION RESULTS

Final Registration Metrics:
  Final mean intensity: 0.5835

Optimized PCA Coefficients (in units of std deviations):
  Mode  1: -0.0846
  Mode  2:  0.5042
  Mode  3:  0.3264
  Mode  4:  0.8800
  Mode  5: -0.4894
  Mode  6: -0.4051
  Mode  7:  1.8894
  Mode  8:  0.7505
  Mode  9:  0.0683
  Mode 10:  0.5504

✓ Registration pipeline complete!


## Save Registration Results


In [12]:
print("\nSaving results...")

# Save final PCA-registered mesh
pca_registered_model_surface.save(str(output_dir / "pca_registered_model_surface.vtk"))
print("  Saved final PCA-registered mesh")

# Save PCA coefficients
np.savetxt(
    str(output_dir / "pca_coefficients.txt"),
    result["pca_coefficients"],
    header=f"PCA coefficients for {len(result['pca_coefficients'])} modes",
)
print("  Saved PCA coefficients")


Saving results...


  Saved final PCA-registered mesh


  Saved PCA coefficients


## Visualize Results


In [13]:
# Create side-by-side comparison
plotter = pv.Plotter(shape=(1, 2), window_size=[1000, 600])

plotter.subplot(0, 0)
plotter.add_mesh(patient_surface, color="red", opacity=1.0, label="Patient")
plotter.add_mesh(
    icp_registered_model_surface, color="green", opacity=1.0, label="ICP Registered"
)
plotter.add_title("ICP Shape Fitting")
plotter.add_axes()

# After PCA shape fitting
plotter.subplot(0, 1)
plotter.add_mesh(patient_surface, color="red", opacity=1.0, label="Patient")
plotter.add_mesh(
    pca_registered_model_surface, color="green", opacity=1.0, label="PCA Registered"
)
plotter.add_title("PCA Shape Fitting")
plotter.add_axes()

plotter.link_views()
plotter.show()

Widget(value='<iframe src="http://localhost:65356/index.html?ui=P_0x1753686ac20_0&reconnect=auto" class="pyvis…